# Input several news articles into the Stardog DNA repositories

There are 7-8 articles from different sources on the topics of the the Trump NY jury conviction (30 May), after-effects of the landslide in Papua New Guinea (26 May), and the FL Supreme Court upholding the 6-week abortion ban (2 April). 

The Trump conviction and FL Supreme Court ruling likely have strong partisan-related support, while reporting on the landslide should show less partisan influence.

Note that a landslide-related WSJ article could not be found in the 25-26 May timeframe.

Make sure that: 
* The DNA flask is running (execute 'flask run' from the dna directory)

In [1]:
import json
import requests

In [2]:
# Delete repositories (if they exist)
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=fl_abortion')
print(response.status_code, response) 
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_trial')
print(response.status_code, response) 
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=landslide')
print(response.status_code, response) 

200 <Response [200]>
200 <Response [200]>
200 <Response [200]>


In [3]:
# Create the repositories for the news articles
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=landslide')
print(response.status_code, response) 
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=fl_abortion')
print(response.status_code, response) 
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_trial')
print(response.status_code, response) 
# Verify that the repositories were created
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories')
print(response.status_code, response.json()) 

201 <Response [201]>
201 <Response [201]>
201 <Response [201]>
200 [{'created': '2024-05-31T17:39:23', 'repository': 'fl_abortion'}, {'created': '2024-05-31T17:39:24', 'repository': 'trump_trial'}, {'created': '2024-05-31T17:39:22', 'repository': 'landslide'}]


In [4]:
# Articles for ingest
landslide_aljazeera = {
            "title": "UN estimates more than 670 killed in Papua New Guinea landslide",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/5/26/un-estimates-more-than-670-killed-in-papua-new-guinea-landslide"}
landslide_breitbart = {
            "title": "UN raises Papua New Guinea landslide death toll estimate to 670",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.breitbart.com/news/un-raises-papua-new-guinea-landslide-death-toll-estimate-to-670/"}
landslide_cnn = {
            "title": "More than 670 people feared dead after Papua New Guinea landslide, UN says",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.cnn.com/2024/05/26/world/papua-new-guinea-landslide-death-toll-intl/index.html"}
landslide_fox = {
            "title": "Hundreds reportedly dead after severe landslide strikes Papua New Guinea village",
            "published": "2024-05-24T00:00:00",
            "url": "https://www.foxnews.com/world/hundreds-feared-dead-severe-landslide-strikes-papua-new-guinea-village"}
landslide_huffpost = {
            "title": "U.N. Migration Agency Estimates More Than 670 Killed In Papua New Guinea Landslide",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.huffpost.com/entry/bc-as-papua-new-guinea-landslide_n_665327a8e4b09d0995d6dd6f"}
landslide_nyt = {
            "title": "Death Toll in Papua New Guinea Landslide Estimated to Be at Least 670",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.nytimes.com/2024/05/26/world/asia/papua-new-guinea-landslide-death.html"}
landslide_watimes = {
            "title": "U.N. estimates over 670 people died in a massive Papua New Guinea landslide as survivors seek safety",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/may/26/un-estimates-over-670-people-died-in-massive-papua/"}
landslide_wsj = dict()


fl_abortion_aljazeera = {
            "title": "Biden denounces Florida abortion ruling as ‘outrageous’ as state vote looms",
            "published": "2024-04-02T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms"}
fl_abortion_breitbart = {
            "title": "Florida Supreme Court Upholds 15-Week Abortion Limit, But Voters Will Decide in November",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.breitbart.com/politics/2024/04/01/florida-supreme-court-upholds-15-week-abortion-limit-but-voters-will-decide-in-november/"}
fl_abortion_cnn = {
            "title": "Florida six-week abortion ban will soon become law, but voters will consider a constitutional amendment this fall",
            "published": "2024-04-02T00:00:00",
            "url": "https://www.cnn.com/2024/04/01/politics/florida-six-week-abortion-ban/index.html"}
fl_abortion_fox = {
            "title": "Florida Supreme Court approves abortion ballot initiative while upholding 15-week ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.foxnews.com/politics/florida-supreme-court-approves-abortion-ballot-initiative-upholding-15-week-ban"}
fl_abortion_huffpost = {
            "title": "Top Florida Court Rules Against Protections For Abortion, Greenlights 6-Week Ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.huffpost.com/entry/florida-court-abortion-6-week-ban_n_65088b87e4b0f04446230d7e"}
fl_abortion_nyt = {
            "title": "Florida Court Allows 6-Week Abortion Ban, but Voters Will Get to Weigh In",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.nytimes.com/2024/04/01/us/florida-abortion-law-supreme-court.html"}
fl_abortion_watimes = {
            "title": "Florida Supreme Court upholds state’s 15-week ban on most abortions, paving way for 6-week ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/apr/1/florida-supreme-court-upholds-states-15-week-ban-2/"}
fl_abortion_wsj = {
            "title": "Florida Abortion Rulings to Keep Issue Front and Center in November",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c"}


trump_trial_aljazeera = {
            "title": "Donald Trump guilty on all counts in hush-money trial, New York jury finds",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/5/30/new-york-jury-finds-donald-trump-guilty-in-new-york-hush-money-trial"}
trump_trial_breitbart = {
            "title": "Guilty on all counts: Trump criminal conviction makes history",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.breitbart.com/news/donald-trump-convicted-on-all-charges-in-hush-money-trial/"}
trump_trial_cnn = {
            "title": "Trump found guilty in hush money trial",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.cnn.com/2024/05/30/politics/donald-trump-hush-money-trial-verdict/index.html"}
trump_trial_fox = {
            "title": "Trump guilty on all counts in New York criminal trial",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.foxnews.com/politics/ny-v-trump-jury-verdict-former-presidents-historic-criminal-trial"}
trump_trial_huffpost = {
            "title": "The Verdict: Inside The Courtroom As Donald Trump Learned He Had Been Convicted",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.huffpost.com/entry/rump-hush-money-courtroom-scene_n_665941fbe4b0b6cf3f47907b"}
trump_trial_nyt = {
            "title": "Trump Convicted on All Counts to Become America’s First Felon President",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.nytimes.com/2024/05/30/nyregion/trump-convicted-hush-money-trial.html"}
trump_trial_watimes = {
            "title": "Trump becomes first former president to be convicted of crime",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/may/30/manhattan-jury-finds-donald-trump-guilty-on-all-co/"}
trump_trial_wsj = {
            "title": "Donald Trump Found Guilty on All 34 Counts in New York Hush-Money Case",
            "published": "2024-05-30T00:00:00",
            "url": "https://www.wsj.com/us-news/law/donald-trump-convicted-87a4e465"}

# repos = ['ceasefire', 'fl_abortion', 'trump_bond', 'trump_trial']
repos = ['fl_abortion', 'landslide', 'trump_trial']
# 2 ~neutral sources (Al Jazeera, WSJ), 3 right/conservative sources(Breitbart, Fox, WA Times)
#    3 left/liberal sources (CNN, Huffington Post, NYT)
sources = {'aljazeera': 'Al Jazeera',    # ~ neutral
           'wsj': 'Wall Street Journal',
           'breitbart': 'Breitbart',     # conservative
           'fox': 'Fox News',
           'watimes': 'Washington Times',
           'cnn': 'CNN',                 # liberal
           'huffpost': 'Huffington Post',
           'nyt': 'New York Times'}

In [5]:
# Function to ingest the first 10 sentences of the 8 news articles regarding a specific topic 
# (ingested to the 'topic' repository)
def ingest_articles(repo: str): 
    for source in sources:
        repo_source = f'{repo}_{source}'
        print('repo_source ', repo_source)
        try:
            with open(f'articles/{repo_source}.txt', 'r') as news:
                news_text = news.read()  
        except:
            print(f'No article text for {repo_source}')
            print()
            continue
        metadata = globals()[repo_source]
        req_dict = metadata
        req_dict["source"] = sources[source]
        req_dict["text"] = news_text
        response = requests.post(
            f'http://127.0.0.1:5000/dna/v1/repositories/narratives?repository={repo}&sentences=10',
            json=req_dict)
        print(response)
        print(response.status_code, response.json())
        print()

In [6]:
ingest_articles('fl_abortion')

repo_source  fl_abortion_aljazeera
<Response [201]>
201 {'narrativeDetails': {'narrativeId': 'd336a6f5', 'narrativeMetadata': {'published': '2024-04-02T00:00:00', 'source': 'Al Jazeera', 'title': 'Biden denounces Florida abortion ruling as ‘outrageous’ as state vote looms', 'url': 'https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms'}, 'numberIngested': 34, 'numberOfSentences': 35, 'numberOfTriples': '899', 'processed': '2024-05-31T17:49:55'}, 'repository': 'fl_abortion'}

repo_source  fl_abortion_wsj
<Response [201]>
201 {'narrativeDetails': {'narrativeId': 'e24d6c3c', 'narrativeMetadata': {'published': '2024-03-25T00:00:00', 'source': 'Wall Street Journal', 'title': 'Florida Abortion Rulings to Keep Issue Front and Center in November', 'url': 'https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c'}, 'numberIngested': 35, 'numberOfSentences': 36, 'numberOfTriples

In [7]:
# Check what articles are in the fl_abortion repository 
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=fl_abortion')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "d336a6f5",
   "narrativeMetadata": {
    "published": "2024-04-02T00:00:00",
    "source": "Al Jazeera",
    "title": "Biden denounces Florida abortion ruling as \u2018outrageous\u2019 as state vote looms",
    "url": "https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms"
   },
   "numberIngested": "34",
   "numberOfSentences": "35",
   "numberOfTriples": "899",
   "processed": "2024-05-31T17:49:55"
  },
  {
   "narrativeId": "e24d6c3c",
   "narrativeMetadata": {
    "published": "2024-03-25T00:00:00",
    "source": "Wall Street Journal",
    "title": "Florida Abortion Rulings to Keep Issue Front and Center in November",
    "url": "https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c"
   },
   "numberIngested": "35",
   "numberOfSentences": "36",
   "numberOfTriples": "939",
   "processed": "2024-05-31T18:01:08"
  },


In [8]:
ingest_articles('landslide')

repo_source  landslide_aljazeera
<Response [201]>
201 {'narrativeDetails': {'narrativeId': '98097564', 'narrativeMetadata': {'published': '2024-05-26T00:00:00', 'source': 'Al Jazeera', 'title': 'UN estimates more than 670 killed in Papua New Guinea landslide', 'url': 'https://www.aljazeera.com/news/2024/5/26/un-estimates-more-than-670-killed-in-papua-new-guinea-landslide'}, 'numberIngested': 18, 'numberOfSentences': 19, 'numberOfTriples': '495', 'processed': '2024-05-31T20:09:25'}, 'repository': 'landslide'}

repo_source  landslide_wsj
No article text for landslide_wsj

repo_source  landslide_breitbart
<Response [201]>
201 {'narrativeDetails': {'narrativeId': '91c84598', 'narrativeMetadata': {'published': '2024-05-26T00:00:00', 'source': 'Breitbart', 'title': 'UN raises Papua New Guinea landslide death toll estimate to 670', 'url': 'https://www.breitbart.com/news/un-raises-papua-new-guinea-landslide-death-toll-estimate-to-670/'}, 'numberIngested': 15, 'numberOfSentences': 16, 'numberOf

In [9]:
# Check what articles are in the landslide repository 
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=landslide')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "98097564",
   "narrativeMetadata": {
    "published": "2024-05-26T00:00:00",
    "source": "Al Jazeera",
    "title": "UN estimates more than 670 killed in Papua New Guinea landslide",
    "url": "https://www.aljazeera.com/news/2024/5/26/un-estimates-more-than-670-killed-in-papua-new-guinea-landslide"
   },
   "numberIngested": "18",
   "numberOfSentences": "19",
   "numberOfTriples": "495",
   "processed": "2024-05-31T20:09:25"
  },
  {
   "narrativeId": "91c84598",
   "narrativeMetadata": {
    "published": "2024-05-26T00:00:00",
    "source": "Breitbart",
    "title": "UN raises Papua New Guinea landslide death toll estimate to 670",
    "url": "https://www.breitbart.com/news/un-raises-papua-new-guinea-landslide-death-toll-estimate-to-670/"
   },
   "numberIngested": "15",
   "numberOfSentences": "16",
   "numberOfTriples": "470",
   "processed": "2024-05-31T20:17:08"
  },
  {
   "narrativeId": "9a9e49ce",
   "narrativeMetadata": {
    "

In [10]:
ingest_articles('trump_trial')

repo_source  trump_trial_aljazeera
<Response [201]>
201 {'narrativeDetails': {'narrativeId': 'eae24e7a', 'narrativeMetadata': {'published': '2024-05-30T00:00:00', 'source': 'Al Jazeera', 'title': 'Donald Trump guilty on all counts in hush-money trial, New York jury finds', 'url': 'https://www.aljazeera.com/news/2024/5/30/new-york-jury-finds-donald-trump-guilty-in-new-york-hush-money-trial'}, 'numberIngested': 50, 'numberOfSentences': 51, 'numberOfTriples': '1331', 'processed': '2024-05-31T23:41:21'}, 'repository': 'trump_trial'}

repo_source  trump_trial_wsj
<Response [201]>
201 {'narrativeDetails': {'narrativeId': '51cdd6b5', 'narrativeMetadata': {'published': '2024-05-30T00:00:00', 'source': 'Wall Street Journal', 'title': 'Donald Trump Found Guilty on All 34 Counts in New York Hush-Money Case', 'url': 'https://www.wsj.com/us-news/law/donald-trump-convicted-87a4e465'}, 'numberIngested': 59, 'numberOfSentences': 60, 'numberOfTriples': '1029', 'processed': '2024-05-31T23:51:12'}, 'repo

In [11]:
# In the trump_trial repository
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=trump_trial')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "eae24e7a",
   "narrativeMetadata": {
    "published": "2024-05-30T00:00:00",
    "source": "Al Jazeera",
    "title": "Donald Trump guilty on all counts in hush-money trial, New York jury finds",
    "url": "https://www.aljazeera.com/news/2024/5/30/new-york-jury-finds-donald-trump-guilty-in-new-york-hush-money-trial"
   },
   "numberIngested": "50",
   "numberOfSentences": "51",
   "numberOfTriples": "1331",
   "processed": "2024-05-31T23:41:21"
  },
  {
   "narrativeId": "51cdd6b5",
   "narrativeMetadata": {
    "published": "2024-05-30T00:00:00",
    "source": "Wall Street Journal",
    "title": "Donald Trump Found Guilty on All 34 Counts in New York Hush-Money Case",
    "url": "https://www.wsj.com/us-news/law/donald-trump-convicted-87a4e465"
   },
   "numberIngested": "59",
   "numberOfSentences": "60",
   "numberOfTriples": "1029",
   "processed": "2024-05-31T23:51:12"
  },
  {
   "narrativeId": "04615e99",
   "narrativeMetadata": {
  

In [12]:
# Dump of the article meta-data and triples
for repo in ('fl_abortion', 'landslide', 'trump_trial'):
    response = requests.get(f'http://127.0.0.1:5000/dna/v1/repositories/narratives?repository={repo}')
    narr_details = response.json()['narratives']
    narr_ids = []
    for detail in narr_details:
        narr_ids.append(detail['narrativeId'])
    for narr_id in narr_ids:
        response = requests.get(
            f'http://127.0.0.1:5000/dna/v1/repositories/narratives/graphs?repository={repo}&narrativeId={narr_id}')
        resp_json = response.json()
        with open(f'dumps/{repo}_{narr_id}_details.json', 'w') as detail_out:
            detail_out.write(json.dumps(dict(resp_json['narrativeDetails'])))
        with open(f'dumps/{repo}_{narr_id}.ttl', 'w') as triples_out:
            for triple in resp_json['triples']:
                triples_out.write(triple)
    